In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_xquad = pd.read_csv("/content/drive/MyDrive/MRC/dataset/xquad.csv")
file_visquad = pd.read_csv("/content/drive/MyDrive/MRC/dataset/visquad.csv")

final_file = pd.concat([file_xquad, file_visquad])
final_file.to_csv("/content/drive/MyDrive/MRC/dataset/fact_checking_file.csv")

**load data**

In [ ]:
link_data = "/content/drive/MyDrive/MRC/dataset/fact_checking_file.csv"

df = pd.read_csv(link_data,index_col=False)
df['claim'] = df[['question','answer']].apply(lambda x : '{} là {}'.format(x[0].replace('?',''),x[1]), axis=1)

In [ ]:
list_title = list(set(df['title'].to_list()))
dict_title = {}

for title in list_title:
    claim = df[df['title'] == title]['claim'].to_list()
    dict_title[title] = claim

**get ner model**

In [ ]:
!pip install transformers
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=57aff110996aae028036c248476c62a5e7219e6bf1aa5ef0271d789e9d0c4661
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/ner-vietnamese-electra-base")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/ner-vietnamese-electra-base")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [ ]:
example = dict_title['Donald Trump'][0]

In [ ]:
example = example + '0.25 + 33'

In [ ]:
from collections import defaultdict

def get_ner(example):
    ner_results = nlp(example)
    ner_dict = defaultdict(list)
    text = ""
    last_label = None
    for i,ner in enumerate(ner_results):
      if "B-" in ner['entity']:
         if i != 0:
           ner_dict[last_label].append(text)
         text = ner['word']
         last_label = ner['entity'][2:]
      if "I-" in ner['entity']:
         text = text + " " + ner['word']

    ner_dict[last_label].append(text)

    return ner_dict

In [ ]:
import pickle

# ner_label = {
#     "PER" : person,
#     "ORG" : org,
#     "LOC" : loc
# }

# with open("/content/drive/MyDrive/largeProject/code/MRC/ner_label.pkl",'wb') as f:
#     pickle.dump(ner_label,f)

In [ ]:
with open("/content/drive/MyDrive/MRC/ner_label.pkl",'rb') as f:
    ner_label = pickle.load(f)

In [ ]:
ner_label.keys()

dict_keys(['PER', 'ORG', 'LOC'])

**regex number**

In [ ]:
import re

def get_number_in_text(text):
    pattern = r'([+-]?[0-9]*\.?[eE]?[0-9]+)'
    matches = re.findall(pattern, text)

    return matches

In [ ]:
get_number_in_text("Trần Hưng Đạo")

[]

**fake module data**

In [ ]:
dict_mapping = {
    "PERSON" : "PER",
    "ORGANIZATION" : "ORG",
    "LOCATION" : "LOC"
}

In [ ]:
import random

def fake_module(text):
    ner_dict = get_ner(text)

    #entity
    flag = False

    key_list = list(ner_dict.keys())
    k = None
    while True:
      k = random.choice(key_list)
      if k != 'MISCELLANEOUS' and k is not None:
        if len(ner_dict[k]) > 0:
          break
      else:
        return text + "là sai"

    v_list = ner_dict[k]
    for v in v_list:
      label = random.choice(ner_label[dict_mapping[k]])
      text = text.replace(v,label)
      if random.random() < 0.3:
          if flag == False:
            flag = True
          else:
            break
    #number
    list_number = get_number_in_text(text)

    if len(list_number) == 0:
       return text

    for num in list_number:
       if random.random() < 0.2:
          if flag == False:
             flag = True
          else: break
       if random.random() < 0.5:
         num_rep = str(random.randint(-10000, 10000))
       else:
         num_rep = str(round(random.uniform(-10000, 10000),2))
       text = text.replace(num,num_rep)

    return text

**Create data**

In [ ]:
question_postfix = ['là bao nhiêu',
                    'như thế nào',
                    'thời gian nào',
                    'lĩnh vực nào',
                    'là ai',
                    'là gì',
                    'nào',
                    'gì'
                  ]

In [ ]:
def replace_postfix(text):
    for ques in question_postfix:
       text = text.replace(ques,'')
    text = text.replace('  ',' ').strip()
    return text

In [ ]:
df['claim'] = df['claim'].map(lambda x : replace_postfix(x))

In [ ]:
df['claim'].to_list()[0]

'Đội thủ Panthers đã thua bao nhiêu điểm là 308'

**create new data**

In [ ]:
evidences = df['answer'].to_list()
claims = df['claim'].to_list()
contexts = df['context'].to_list()
titles = df['title'].to_list()

In [ ]:
new_claims = []
new_contexts = []
new_evidences = []
labels = []

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(evidences))):
  try:
    # 0: SUPPORTED, 1: NEI, 2: REFUTED

    #supported
    new_claims.append(claims[i])
    new_contexts.append(contexts[i])
    new_evidences.append(evidences[i])
    labels.append(0)

    #nei
    while True:
      title = random.choice(list_title)
      if title != titles[i]:
        break
    new_claims.append(random.choice(dict_title[title]))
    new_contexts.append(contexts[i])
    new_evidences.append(evidences[i])
    labels.append(1)

    #refuted
    new_claims.append(fake_module(claims[i]))
    new_contexts.append(contexts[i])
    new_evidences.append(evidences[i])
    labels.append(2)

  except:
    continue

100%|██████████| 33468/33468 [1:36:16<00:00,  5.79it/s]


In [ ]:
df_fcd = {
    "context" : new_contexts,
    "claim" : new_claims,
    "evidence" : new_evidences,
    "label" : labels
}

In [ ]:
df_fcd = pd.DataFrame(df_fcd)

In [ ]:
df_fcd.head(10)

,context,claim,evidence,label
0,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Đội thủ Panthers đã thua bao nhiêu điểm là 308,308,0
1,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Tại sao Zopyrion vây hãm Olbia là Zopyrion nhậ...,308,1
2,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Đội thủ Panthers đã thua bao nhiêu điểm là 5778,308,2
3,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Jared Allen có bao nhiêu lần vật ngã trong sự ...,136,0
4,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Các thành thị của Thuỵ Sĩ thiết lập hiến pháp ...,136,1
5,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Jared Allen có bao nhiêu lần vật ngã trong sự ...,136,2
6,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Luke Kuechly đã có bao nhiêu cú húc là 118,118,0
7,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Những người da đỏ trong các bộ phim về người M...,118,1
8,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Luke Kuechly đã có bao nhiêu cú húc là -572,118,2
9,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",Josh Norman đã đoạt bóng được bao nhiêu lần là...,bốn lựa chọn,0


In [ ]:
df_fcd.to_csv('/content/drive/MyDrive/MRC/dataset/fact_checking_dataset.csv')

**module pharaphase (optional)**

In [ ]:
!pip install simplet5

In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.load_model("t5", "SCM-LAB/vietnamese_question_paraphrasing_ViT5_base", use_gpu=True)

In [ ]:
question = "Tên gọi được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm là gì"
model.predict(f"question paraphrasing: {question}", num_return_sequences= 2, num_beams= 5)